In [1]:
import os
import findspark

from alkansya.data_generators import generate_ohlcv
from alkansya.utils import get_configurations

from pyspark.sql import SparkSession
from pyspark.sql import functions as f

import pandas as pd
from datetime import datetime

os.environ["ENV"] = "DEV"
findspark.init()

cfg = get_configurations()

PATH_TO_BRONZE = cfg["path_to_bronze"]
PATH_TO_SILVER = cfg["path_to_silver"]
PATH_TO_GOLD = cfg["path_to_gold"]

spark = SparkSession.builder.master("local[*]").getOrCreate()

23/12/30 01:08:00 WARN Utils: Your hostname, Nigels-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.254.124 instead (on interface en0)
23/12/30 01:08:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/30 01:08:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
dfs = spark.read.parquet(f"{PATH_TO_GOLD}/cleaned")

for resolution in [15,30,60,1440]:
    generate_ohlcv(dfs, resolution, PATH_TO_GOLD)

In [5]:
dfs = spark.read.parquet(f"{PATH_TO_GOLD}/resolution_1440min")
dfs.show()

+-------------------+-------+-------+-------+-------+--------+-------------+
|               time|   open|   high|    low|  close|  volume|currency_pair|
+-------------------+-------+-------+-------+-------+--------+-------------+
|2019-01-09 08:00:00|1.27576|1.27797|1.27066|1.27395| 91172.0|       GBPUSD|
|2019-01-10 08:00:00|1.27418|1.28035|1.27156|1.27889|161779.0|       GBPUSD|
|2019-01-11 08:00:00|1.27868|1.27939|1.27279|1.27607|141804.0|       GBPUSD|
|2019-01-14 08:00:00|1.28533|1.28639|1.28404|1.28508| 23999.0|       GBPUSD|
|2019-01-15 08:00:00|  1.285|1.29303|1.28184|1.29135|124252.0|       GBPUSD|
|2019-01-16 08:00:00|1.29148|1.29148|1.26679|1.28681|163631.0|       GBPUSD|
|2019-01-17 08:00:00| 1.2867| 1.2897|1.28405|1.28753|130260.0|       GBPUSD|
|2019-01-18 08:00:00|1.28781|1.30005| 1.2832|1.29756|140608.0|       GBPUSD|
|2019-01-21 08:00:00|1.28768|1.28792|1.28476|1.28695| 23695.0|       GBPUSD|
|2019-01-22 08:00:00|1.28682|1.29103|1.28301|1.28731|110491.0|       GBPUSD|

In [8]:
from alkansya.feature_engineering import spark_query as query

dfs = spark.read.parquet(f"{PATH_TO_GOLD}/resolution_60min")

dfs_temp = dfs.select(
    "*",
    query.simple_moving_average(window_size_days=3),
    query.exponential_moving_average(window_size_days=3),
    query.volatility(window_size_days=3),
    # *query.support_and_resistance(window_size_days=28, prominence=0.001, wlen=14, plateau_size=3)
)

dfs_temp.show()

+-------------------+-------+-------+-------+-------+------+-------------+------------------+------------------+--------------------+-----------+-----------+-------------+-------------+
|               time|   open|   high|    low|  close|volume|currency_pair|         sma_3_day|         ema_3_day|    volatility_3_day|upper_bound|lower_bound| peak_indeces|  dip_indeces|
+-------------------+-------+-------+-------+-------+------+-------------+------------------+------------------+--------------------+-----------+-----------+-------------+-------------+
|2019-01-07 15:00:00|0.71342|0.71349|0.71331|0.71336| 348.0|       AUDUSD|0.7133600115776062|              NULL|                NULL|    0.71342|    0.71336|           []|           []|
|2019-01-07 16:00:00| 0.7134|0.71401|0.71318|0.71335|3762.0|       AUDUSD|0.7133550047874451|0.7133540034294129|3.540335274915098...|     0.7134|    0.71335|           []|           []|
|2019-01-07 17:00:00|0.71345|0.71388|0.71258|0.71349|4598.0|       AUD

In [1]:
dfs.withColumn("maximum_swing_per_day", (1-f.col("low")/f.col("high"))).orderBy(f.col("maximum_swing_per_day").desc()).filter(f.col("time")>"2021-01-01").show()

NameError: name 'dfs' is not defined

In [24]:
df = pd.read_parquet(f"{PATH_TO_GOLD}/resolution_1440min")
df['theoretical_max_swing'] = 1-df["low"]/df["high"]
df['theoretical_swing_open_close'] = df["open"]/df["close"]

# Exclude 2020 pandemic start
df = df[df["time"]>datetime(2021, 1,1)].sort_values("theoretical_swing_open_close", ascending=False)